In [1]:
import torch
import torchvision
import numpy as np
from torch import nn
from torch.nn import functional as F
from sklearn.preprocessing import StandardScaler

torch.manual_seed(0)

# Load in the resnet18 architecture
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights ='ResNet18_Weights.DEFAULT')


c:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0


In [2]:
# Get only the embedding layers of ResNet
modules = list(resnet.children())[:9]
feature_extractor = nn.Sequential(*modules)
for f in feature_extractor.parameters():
  f.requires_grad = False

In [3]:
# Setup device for model training and evaluation
device = 'cuda' if torch.cuda.is_available else 'cpu'
print("Running on device:", device)
if device == 'cpu': "Warning: running on CPU may be slow"

Running on device: cuda


In [4]:
# Define the classifier, using the pretrained ResNet feature-extractor
# as a sktarting point.
class LeukemiaNet_Features(nn.Module):
  def __init__(self) -> None:
    super(LeukemiaNet_Features, self).__init__()
    self.embeddings = feature_extractor
    self.name = "LeukemiaNet Feature Extractor"
  def forward(self, x):
    # Compute embeddings:
    return self.embeddings(x.unsqueeze(0))

In [5]:
def get_embeddings(dataset, device : str):
  feature_extractor = LeukemiaNet_Features().to(device)
  feature_extractor.eval()
  with torch.no_grad():
    counts = {0:1, 1:1, 2:1, 3:1, 4:1}
    for sample, label in dataset:
      elem = sample.to(device)
      emb = feature_extractor(elem)
      emb = emb.detach().cpu().flatten().numpy()
      np.savetxt("./Embeddings/%s/%d.txt"%(dataset.classes[label], counts[label]), emb)
      counts[label] += 1

In [6]:
images = torchvision.datasets.ImageFolder('./Data/', transform=torchvision.transforms.ToTensor())

In [7]:
get_embeddings(images, device)